# 1. Daten auslesen und formatieren

In [ ]:
# Importiere die benötigten Bibliotheken
import pandas as pd
import sqlite3

# 1. CSV-Datei laden
csv_file = "Daten.csv"
df = pd.read_csv(csv_file, sep=';', decimal=',')  # Passe Trennzeichen und Dezimalzeichen an

# Ausgabe der Anzahl der Zeilen
print(f"Anzahl der Zeilen: {len(df)}")

# Ausgabe der Spaltentitel
print(f"Spaltentitel: {list(df.columns)}")

# Ausgabe des Formats der Datenspalten
print("Format der Datenspalten:")
print(df.dtypes)

# 2. Verbindung zur SQLite-Datenbank herstellen (oder erstellen, falls nicht vorhanden)
db_file = "Test.db"
conn = sqlite3.connect(db_file)

# 3. Daten in die Datenbank schreiben
table_name = "Daten"
df.to_sql(table_name, conn, if_exists='replace', index=False)
print(f"Daten wurden in die Tabelle '{table_name}' geschrieben.")

# 4. Verbindung schließen
conn.close()

Anzahl der Zeilen: 1000000
Spaltentitel: ['TimeStamp', 'ActualPos', 'ActualForce', 'Nahtnummer', 'LaserPwr', 'Befehl']
Format der Datenspalten:
TimeStamp       object
ActualPos      float64
ActualForce      int64
Nahtnummer      object
LaserPwr         int64
Befehl          object
dtype: object
